# Notebook for asking question to friends

In [ ]:
import requests
import json
import os
import openai
import ls_nb_utils
import pandas as pd
# from geopy import distance
import certifi
import logging
from http.client import HTTPConnection
from importlib import reload
import copy
import math


In [ ]:
os.path.abspath('.')

In [ ]:
# pip install openai

In [ ]:
# import geopy
# geopy.__version__

In [ ]:
reload(ls_nb_utils)

In [ ]:
openai.api_key  = os.environ['OPENAI_API_KEY']
client = openai.OpenAI()


In [ ]:
ls_app_url =  'https://pocketverse.herokuapp.com/LS_API'
ls_app_username = os.environ['LS_USERNAME']
ls_app_password = os.environ['LS_PASSWORD']
ls_app_username, ls_app_password

In [ ]:
resp_app = ls_nb_utils.get_access_token(ls_app_username, ls_app_password, 'jwt_login', ls_app_url)
resp_app

In [ ]:
token=resp_app['token']

In [ ]:
tok_decoded = ls_nb_utils.decode_jwt(token)
tok_decoded

In [ ]:
uid = tok_decoded['payload']['userId']
uid

### Check API

In [ ]:
import datetime
import time

In [ ]:
# https://linkedspaces.atlassian.net/l/cp/Q99Ti6qf

In [ ]:
timestamp = 1661256000000
route = 'lsagent/user/places'
categories = [
      'restaurant', 
      'hotel', 
      'lodging', 
      'attraction', 
      'sightseeing', 
      'golf', 
      'cafe', 
      'grocery', 
      'shopping', 
      'airport', 
      'transportation',
      'activity']
cat_short = ['golf', 'cafe', 'restaurant']

location_sj = {
    "center": {
        "latitude": 37.335480,
        "longitude": -121.893028
    },
    "radius": 10000.0
}

location_sf = {
        "center": {
          "latitude": 37.7937,
          "longitude": -122.3965
        },
        "radius": 100000.0
    }
    
params = {
  "userName": "inseo",
  "filters": {
    "location": location_sj,
    "categories": cat_short,
    "time": {
      "timestamp": timestamp, 
      "window": 2000
    }
  }
}

In [ ]:
print (params)

In [ ]:
reload(ls_nb_utils)

In [ ]:
res = ls_nb_utils.test_ls_post(params, route, url = ls_app_url, token = token, target = 'packet')

In [ ]:
res

In [ ]:
len(res['placeList'])

### Structure The question

In [ ]:
td = datetime.datetime.today()
td = datetime.date.isoformat(td)

In [ ]:
system_message = f'''Idendtify what the user is asking about by assigning one of the categories from the following list
[{categories}]. Identify the name of geographical entity the user is asking about. Identify the year the user asking about. Estimate the approximate
area in squared miles of the identified geographical entity and assign the number to the variable named Area
'''
system_message

In [ ]:
report_message = '''Present the input as JSON object according to the following schema:
{ category: 'shopping', location: { 
"name": name, "center": {latitude: value, longitude: value}, 
area:value (miles**2)}, time: YYYY-MM }
'''

In [ ]:
user_text = f''' What's the name of a Korean restaurant I visited in San Francisco two year ago. Today is {td}'''
user_text = f''' What's the name of the airport I visited in San Francisco five years ago. Today is {td}'''
user_text = f'''What restaurant did I visit this year in Santa Clara? Today is {td}'''
# user_text = f''' What's the name of the airport I visited in Europe five years ago. Today is {td}'''

In [ ]:
%%time
obj, text = ls_nb_utils.req_ai2(client, user_text, model_name_system = 'gpt-3.5-turbo', 
                    system_message = system_message, report_message = report_message)

In [ ]:
obj, text

### Building a filter

In [ ]:
ts = datetime.datetime.fromisoformat(obj['time']+'-01').timestamp()*1000

In [ ]:
center = obj['location']['center']
radius2 = obj['location']['area']
radius = math.sqrt(radius2)
METERS_IN_A_MILE = 1609.34
radius = radius*METERS_IN_A_MILE

In [ ]:
params = {
  "userName": "inseo",
  "filters": {
    "location": {'center':center, 
                 'radius':radius
                },
    "categories": [obj['category']],
    "time": {
      "timestamp": ts, 
      "window": 365
    }
  }
}

In [ ]:
res = ls_nb_utils.test_ls_post(params, route, url = ls_app_url, token = token, target = 'packet')

In [ ]:
res

In [ ]:
params

In [ ]:
list_of_places = res['placeList']

### Finding a match

In [ ]:
%%time
system_message = f'''find two locations from this list {list_of_places} that match best the user's request. 
                     Report placeIndex and placeName for the found match'''
report_message = '''Present input as JSON object according to the following schema:
{"matching_places":[{"placeIndex": 27, "placeName": "Starbucks"}]}'''

obj, text = ls_nb_utils.req_ai2(client, user_text, model_name_system = 'gpt-3.5-turbo', 
                    system_message = system_message, report_message = report_message)

In [ ]:
text

In [ ]:
obj

In [ ]:
system_message

